In [ ]:
import glob
import subprocess

catalog = "air_polution_analytics_dev"
bronze_schema = "01_bronze"
landing_schema = "00_landing"
files_to_process = 100

landing_dir = f"/Volumes/{catalog}/{landing_schema}/openaq/measurements"

skip_files = []
if spark.catalog.tableExists(f"{catalog}.{bronze_schema}.air_quality_measurements"):
    df = spark.read.table(f"{catalog}.{bronze_schema}.air_quality_measurements")
    skip_files = [str(f.source_file_name) for f in df.select("source_file_name").distinct().collect()]

print(f'found {len(skip_files)} files to skip')

datastore_path = dbutils.widgets.get("datastore_path")
files = [f.split('/').pop().replace('.zip', '') for f in glob.glob(f"{datastore_path}/[0-9]*.json.zip")]
print(f'found {len(files)} files')

files = list(filter(lambda f: f not in skip_files, files))
files.sort()    # To guarantee the order of ingested records

print(f'found {len(files)} files to ingest')

files = [f"{datastore_path}/{f}.zip" for f in files]

to_delete = glob.glob(f"{landing_dir}/*.json")
if to_delete:
    subprocess.run(["rm"] + to_delete)

for f in files[0:files_to_process]:
    subprocess.run(f"unzip -o -d{landing_dir} {f}".split(" "), stdout=subprocess.DEVNULL)

print(f"ingesting {len(files[0:files_to_process])} files")